In [37]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [38]:
# Загрузка и предобработка данных
iris = load_iris()
X = iris.data
y = iris.target

In [39]:
# Кодирование меток классов
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y.reshape(-1, 1))

In [40]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
# Функция для создания и обучения модели
def create_and_train_model(hidden_neurons):
    model = Sequential([
        Dense(hidden_neurons, input_shape=(4,), activation='relu'),
        Dense(y.shape[1], activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=500, validation_split=0.2, callbacks=[early_stopping], verbose=0)

    return model

In [42]:
# Поиск минимального количества нейронов для достижения accuracy = 1
min_neurons = 1
max_accuracy = 0

In [43]:
while not np.isclose(max_accuracy, 1.0, atol=1e-7):
    model = create_and_train_model(min_neurons)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)

    # Обновление max_accuracy, если она близка к 1.0
    if np.isclose(accuracy, 1.0, atol=1e-7):
        max_accuracy = accuracy
    else:
        max_accuracy = max(max_accuracy, accuracy)
        min_neurons += 1

In [44]:
print(f'Количество нейронов: {min_neurons}\n'
      f'Точность: {max_accuracy}')

Количество нейронов: 2
Точность: 1.0


In [45]:
# Вывод описания модели
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 2)                 10        
                                                                 
 dense_37 (Dense)            (None, 3)                 9         
                                                                 
Total params: 19 (76.00 Byte)
Trainable params: 19 (76.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
